In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('mart_overall.csv')
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,days_to_delivery,estimated_days_to_delivery,...,seller_region,season,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,customer_region,distance
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,15,...,남동부,rainy,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.576983,-46.587161,남동부,18.566632
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,17,...,남동부,dry,af07308b275d755c9edb36a90c618231,47813,barreiras,TO,-12.135259,-45.005406,북부,856.076259
2,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,17,...,남동부,dry,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.945892,-38.456194,북동부,961.437029
3,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,26,...,남동부,dry,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.745150,-48.514783,중서부,512.100044
4,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,26,...,남동부,rainy,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,-5.774190,-35.271143,북동부,1816.085655


In [4]:
from pyspark.sql import SparkSession
# Spark 세션 생성
spark = SparkSession.builder.appName("olist").getOrCreate()

spk_orders = spark.read.csv('mart_orders.csv', header=True, inferSchema=True)
spk_orders.createOrReplaceTempView("orders")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 13:46:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/26 13:46:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/26 13:46:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/02/26 13:46:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/02/26 13:46:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
spk_reviews = spark.read.csv('review_data.csv', header=True, inferSchema=True)
spk_reviews.join(spk_orders, spk_reviews.order_id == spk_orders.order_id, 'inner').createOrReplaceTempView("reviews")
spk_reviews.show()

+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+-----------+
|_c0|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|sentimental|
+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+-----------+
|  9|8670d52e15e00043a...|b9bf720beb4ab3728...|           4|           recomendo|  aparelho eficient...| 2018-05-22 00:00:00|    2018-05-23 16:45:47|    Neutral|
| 15|3948b09f7c818e2d8...|e51478e7e277a8374...|           5|     Super recomendo|  vendedor confiáve...| 2018-05-23 00:00:00|    2018-05-24 03:00:01|   Positive|
| 19|373cbeecea8286a2b...|583174fbe37d3d5f0...|           1|Não chegou meu pr...|               péssimo| 2018-08-15 00:00:00|    2018-08-15 04:10:37|   Negative|
| 22|d21bbc789670eab77...|4f

25/02/26 13:46:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp, sentimental
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp, sentimental
Expected: _c0 but found: 
CSV file: file:///app/spark/Olist/review_data.csv


In [6]:
df = spk_orders.join(spk_reviews, 'order_id', 'inner') \
               .select('order_id', 'seller_id', 'seller_region', 'customer_id', 'distance','estimated_days_to_delivery', 
                       'seller_state', 'customer_state','customer_region', 'days_to_delivery','delay_flag_estimated','delay_flag_maen', 'review_score','sentimental','review_comment_message')
df.show()

+--------------------+--------------------+-------------+--------------------+------------------+--------------------------+------------+--------------+---------------+----------------+--------------------+---------------+------------+-----------+----------------------+
|            order_id|           seller_id|seller_region|         customer_id|          distance|estimated_days_to_delivery|seller_state|customer_state|customer_region|days_to_delivery|delay_flag_estimated|delay_flag_maen|review_score|sentimental|review_comment_message|
+--------------------+--------------------+-------------+--------------------+------------------+--------------------------+------------+--------------+---------------+----------------+--------------------+---------------+------------+-----------+----------------------+
|53cdb2fc8bc7dce0b...|289cdb325fb7e7f89...|       남동부|b0830fb4747a6c6d2...|  856.076259389407|                        17|          MG|            TO|           북부|              12|       

In [7]:
from pyspark.sql.functions import col, when, count, isnan, isnull, mean, lit, sum, avg, round
df = df.withColumn("same_state", when(col("seller_state") == col("customer_state"), "Yes").otherwise("No"))

In [8]:
# Create a temporary view from the DataFrame
df.createOrReplaceTempView("df_view")

query = """
    SELECT sentimental, AVG(review_score) as avg_review_score,COUNT(DISTINCT order_id) as cnt
    FROM df_view
    GROUP BY sentimental
    ORDER BY sentimental
"""
spark.sql(query).show()

+-----------+------------------+----+
|sentimental|  avg_review_score| cnt|
+-----------+------------------+----+
|   Negative|1.9561128526645768| 565|
|    Neutral|3.6842105263157894|5566|
|   Positive| 4.770824699544136|4380|
+-----------+------------------+----+



- comment에 배송과 관련된 정보가 있을 때, 리뷰 점수?
- WHERE review_comment_message REGEXP "antes"

In [9]:
query = """
    SELECT sentimental, AVG(review_score) as avg_review_score,COUNT(DISTINCT order_id) as cnt
    FROM df_view
    WHERE review_comment_message REGEXP "antes"
    GROUP BY sentimental
    ORDER BY sentimental
"""
spark.sql(query).show()

+-----------+------------------+---+
|sentimental|  avg_review_score|cnt|
+-----------+------------------+---+
|   Negative|3.2083333333333335| 20|
|    Neutral| 4.543543543543543|608|
|   Positive| 4.876744186046512|804|
+-----------+------------------+---+

